## Data processing 

In [16]:
import pandas as pd
df = pd.read_csv('data/winning_candidates_2070.csv')
# df = df.drop(['Title', 'Sector Name', 'Funding Agencies', 'Source URLs'], axis=1)
df

,District,Zone,Geographical Region,Development Region,S.no,Const,Candidate Name,Party Name,Votes Secured,Remark,Total Votes,Percentage of Votes,Victory Margin,Cumulative Victory Margin (total),Cumulative Victory Margin (average),Seat share,Unnamed: 16,Unnamed: 17
0,Taplejung,Mechi,Mountain,East,1,1,Bhupendra Thebe,Communist Party of Nepal (UML),7434,Elected,24093,"30,86%","0,58%","12,08%","6,33%",1,NaN,NaN
1,Taplejung,Mechi,Mountain,East,1,1,Lila Devi Bokhim Limbu,Nepali Congress (NC),7295,Not Elected,24093,"30,28%",NaN,NaN,NaN,NaN,NaN,NaN
2,Taplejung,Mechi,Mountain,East,2,2,Dambardhoj Tumbahamphe,Communist Party of Nepal (UML),7034,Elected,18774,"37,47%","11,51%",NaN,NaN,NaN,NaN,NaN
3,Taplejung,Mechi,Mountain,East,2,2,Pratap Prakash Hangam,Nepali Congress (NC),4874,Not Elected,18774,"25,96%",NaN,NaN,NaN,NaN,NaN,NaN
4,Panchthar,Mechi,Hill,East,3,1,Ganesh Kumar Kambang,Communist Party of Nepal (UML),13082,Elected,36226,"36,11%","4,32%","-3,50%","-3,50%","0,5",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,Kanchanpur,Mahakali,Terai,Far-Western,237,2,Tara Lama Tama,Communist Party of Nepal (UML),9676,Not Elected,44550,"21,72%",NaN,NaN,NaN,NaN,NaN,NaN
476,Kanchanpur,Mahakali,Terai,Far-Western,238,3,Bahadur Singh Thapa,Nepali Congress (NC),12836,Elected,35978,"35,68%","8,87%",NaN,NaN,NaN,NaN,NaN
477,Kanchanpur,Mahakali,Terai,Far-Western,238,3,Nar Bahadur Dhami,Communist Party of Nepal (UML),9643,Not Elected,35978,"26,80%",NaN,NaN,NaN,NaN,NaN,NaN
478,Kanchanpur,Mahakali,Terai,Far-Western,239,4,Ramesh Lekhak,Nepali Congress (NC),14023,Elected,33011,"42,48%","26,01%",NaN,NaN,NaN,NaN,NaN
